In [ ]:
from dotenv import load_dotenv

load_dotenv()

## Recursive URL loader

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders.recursive_url_loader import RecursiveUrlLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.memory import ConversationBufferWindowMemory

In [ ]:
url = "https://python.langchain.com/docs/modules/"
loader = RecursiveUrlLoader(url=url)
docs = loader.load()

In [ ]:
len(docs)

In [ ]:
docs[0].metadata

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)
documents = text_splitter.split_documents(docs)

In [ ]:
len(documents)
index_name = "langchain_docs_index"
embeddings = OpenAIEmbeddings()

In [ ]:
index_name = "langchain_docs_index"
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(docs, embeddings)
vectorstore.save_local(index_name)

In [ ]:
memory = ConversationBufferWindowMemory(k=2)
new_vectorstore = FAISS.load_local(index_name, embeddings)
qa = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(
        model="gpt-3.5-turbo-16k",
        temperature=0,
        streaming=True,
        callbacks=[StreamingStdOutCallbackHandler()],
    ),
    memory=memory,
    chain_type="stuff",
    retriever=new_vectorstore.as_retriever(),
)

In [ ]:
qa.run("What can I do with langchain?")

In [ ]:
qa.run("How can I run the new OpenaAI functions?")